In [1]:
library(tidyverse)
library(dplyr)
library(stringr)
library(readr)
library(data.table)

UBC_ISD <- fread("UBC-ISD.csv", encoding = "UTF-8")

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'data.table'


The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:dplyr':

    between, first, last


The following object is masked from 'package:purrr':

    transpose




In [2]:
#selecting quote tweets and tweets
twitter_data <- UBC_ISD |>
select("quoted.twitter.tweet/text", "twitter.tweet/text")|>
rename(quoted_text="quoted.twitter.tweet/text")|>
rename(text="twitter.tweet/text")

twitter_data <- tibble(text = c(twitter_data$quoted_text, twitter_data$text))

count(twitter_data)

In [10]:

users <- c("adriandix", "AlikaMD", "AmyGreerKalisz", "angie_rasmussen", "AnnaBlakney", 
           "AntibioticDoc", "asapscience", "ASPphysician", "atRachelGilmore", "binhanv", 
           "BirinderNarang", "blackdocscanada", "BogochIsaac", "bornk", "carlyweeks", 
           "CaulfieldTim", "CDCofBC", "cdube_sante", "cfpcceo", "ChiefSciCan", "cmcovidtf", 
           "CMOH_Alberta", "CMOH_NL", "conquercovid19", "COVID_19_Canada", "COVIDSciOntario", 
           "CPHO_Canada", "ctouzin", "CTV_AvisFavaro", "DeNovo_Fatima", "deonandan", "drfisman", 
           "Dr_ChrisSimpson", "drgigiosler", "DrKaliBarrett", "drmwarner", "drsusanshaw", 
           "DrVivianS", "egpayne", "epdevilla", "ErnieHudsonPEI", "everetthindley", "First10EM", 
           "GermHunterMD", "glenpyle", "heysciencesam", "hgagneTVA", "IDEpiPhD", "imgrund", 
           "iPreetBrar", "IrfanDhalla", "j_mcelroy", "jasonfherring", "jfrketich", "jkwan_md", 
           "Johnrockdoc", "JuliaWongCBC", "juliegreenMLA", "Justin_Ling", "jyangstar", "KashPrime", 
           "KatharineSmart", "Kevin_Parent", "KindrachuckJason", "KrishanaSankar", "kwadwo777", 
           "LaurenPelley", "LisaBarrettID", "McGillOSS", "MerrimanPaul", "MichaelSchwandt", 
           "MLAStefanson", "moirawyton", "moriartylabs", "MPaiMD", "NaheedD", "NathanStall", 
           "NightShiftMD", "NoLore", "OttawaHealth", "paimadhu", "PattyHajdu", "picardonhealth", 
           "RicharLisa", "roussin_brent", "sabaeitizaz", "sabiVM", "SammyG_MD", "sarperotto", 
           "SciChefCan", "sciencemonkeyca", "ScienceUpFirst", "sdbaral", "shandro", "SharkawyMD", 
           "shazmamithani", "ShephardDorothy", "srinmurthy99", "SteiniBrown", "theresaboyle", 
           "thisisourshotca", "TorontoIDDOC", "UbakaOgbogu", "VaxHuntersCan", "VeraEtches", 
           "VikCBC", "wickdchiq", "zachchurchhill", "zchangla", "DrKathleenRoss1")

#calculating tweet counts and percentages
mention_counts <- data.frame(user = users) |>
  rowwise() |>
  mutate(mentions = sum(str_detect(twitter_data, paste0("@", user)))) |>
  ungroup()|>
  mutate(percentage=mentions/8441988*100)

mention_counts

: 